In [3]:
import pandas as pd

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [7]:
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

# 1. LLM 기반 질의 생성

이번 실습에서는 총 30개의 QA 데이터 셋을 만들어보겠습니다.

먼저 LLM 기반 질의 생성을 통해 15개의 QA 데이터 셋을 만들어보겠습니다.

15 + 10 + 5 = 30

# 2. Human in the Loop 실습

10개의 QA 데이터 셋을 만들어보겠습니다.

# 3. 이미 FaQ가 있는 경우
5개의 FaQ를 이용해서 만들어보겠습니다 !

qid, query, generation_gt, retrieval_gt를 만들어서 넣어줘야 합니다

FaQ가 있으니, query와 generation_gt는 이미 있습니다.
qid와 retrieval_gt는 직접 만들어주셔야 합니다.

In [18]:
question = [
    "",
    "",
    "",
    "",
    "",
]

In [23]:
answer = [
    "a",
    "b",
    "c",
    "d",
    "e",
]

In [24]:
from uuid import uuid4

qid = [str(uuid4()) for _ in range(len(question))]

In [25]:
generation_gt = [[a] for a in answer]

In [26]:
generation_gt

[['a'], ['b'], ['c'], ['d'], ['e']]

이제 retrieval_gt 만 남았습니다

retrieval_gt를 직접 만드는 방법에는 크게 3가지가 있습니다.

1. 직접 찾기
2. 검색 시스템을 만들어 조금 쉽게 찾아보기

하는 김에 2가지 방법을 모두 사용해보겠습니다.